## Setup

In [ ]:
# Install TensorFlow Dececision Forests and the dependencies used in this colab.
!pip install tensorflow_decision_forests plotly scikit-learn wurlitzer -U -qq

In [ ]:

import tensorflow_decision_forests as tfdf

import matplotlib.colors as mcolors
import math
import os
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from plotly.offline import iplot
import plotly.graph_objs as go

## create random dataset

In [ ]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt


# Set random seed for reproducibility
np.random.seed(42)

# Define parameters for the four different settings
settings = [
    {"mean": [1, 2, 3, 4, 5, 6, 7, 8], "covariance": np.eye(8)},
    {"mean": [10, 9, 8, 7, 6, 5, 4, 3], "covariance": np.diag([1, 2, 3, 4, 5, 6, 7, 8])},
    {"mean": [5, 5, 5, 5, 5, 5, 5, 5], "covariance": np.ones((8, 8)) * 0.5},
    {"mean": [0, 1, 0, 1, 0, 1, 0, 1], "covariance": np.array([[1, 0, 0, 0, 0, 0, 0, 0],
                                                                [0, 2, 0, 0, 0, 0, 0, 0],
                                                                [0, 0, 3, 0, 0, 0, 0, 0],
                                                                [0, 0, 0, 4, 0, 0, 0, 0],
                                                                [0, 0, 0, 0, 5, 0, 0, 0],
                                                                [0, 0, 0, 0, 0, 6, 0, 0],
                                                                [0, 0, 0, 0, 0, 0, 7, 0],
                                                                [0, 0, 0, 0, 0, 0, 0, 8]])}
]

# Initialize an empty list to store generated data
data_points = []

# Generate 500 data points for each parameter setting
for setting in settings:
    mean_vector = setting["mean"]
    covariance_matrix = setting["covariance"]

    # Generate 500 data points with multivariate normal distribution
    data = np.random.multivariate_normal(mean_vector, covariance_matrix, size=500)

    # Append the generated data to the list
    data_points.append(data)

# Concatenate the data points along the first axis to create the final dataset
final_dataset = np.concatenate(data_points, axis=0)

# Print the shape of the final dataset
print("Shape of the final dataset:", final_dataset.shape)

final_dataset= pd.DataFrame(final_dataset)
deneme = final_dataset

In [ ]:
final_dataset['class'] = 1
final_dataset



In [ ]:
column_names = deneme.columns.tolist()
new_column_names = {
    column_names[0]: 'NewFeature1',
    column_names[1]: 'NewFeature2',
    column_names[2]: 'NewFeature3',
    column_names[3]: 'NewFeature4',
    column_names[4]: 'NewFeature5',
    column_names[5]: 'NewFeature6',
    column_names[6]: 'NewFeature7',
    column_names[7]: 'NewFeature8',


}

deneme = deneme.rename(columns=new_column_names)
deneme.dtypes
deneme.columns.astype(str)

In [ ]:
deneme

In [ ]:
from sklearn.cluster import KMeans, AgglomerativeClustering

kmeans = KMeans(n_clusters=4, random_state=0)
clusters_kmeans = kmeans.fit_predict(deneme)
clusters_kmeans

In [ ]:
plt.subplot(132)
plt.scatter(deneme['NewFeature4'], deneme['NewFeature8'], c=clusters_kmeans, cmap='viridis')
plt.title("K-Means Clustering")

In [ ]:
def mean(deneme, clusters_list):
    cluster_means = []
    for cluster_index in clusters_list:
        cluster_data = deneme.iloc[cluster_index]
        mean = np.mean(cluster_data, axis=0)
        cluster_means.append(mean)
    return cluster_means

In [ ]:
kmeans_means = mean(deneme, clusters_kmeans)

In [ ]:
kmeans_means

In [ ]:
kmeans_means[-1]

In [ ]:
kmeans_means[-501]

In [ ]:
kmeans_means[-1001]

In [ ]:
kmeans_means[-1501]

In [ ]:
kmeans_covariances

In [ ]:
import numpy as np

# Assuming 'original_data' is your original dataset with shape (n_samples, n_features)
# Replace this with your actual dataset
np.random.seed(42)
original_data = final_dataset

# Function to permute each variable independently
def permute_variable(variable):
    return np.random.permutation(variable)

# Permute each variable in the original data independently
synthetic_data = np.apply_along_axis(permute_variable, axis=0, arr=original_data)

# Print the first few rows of the synthetic data
print("Synthetic Data:")
print(synthetic_data[:5, :])
synthetic_data= pd.DataFrame(synthetic_data)
synthetic_data

In [ ]:
synthetic_data['class'] = 2
synthetic_data

In [ ]:
df_combined = pd.concat([final_dataset, synthetic_data], ignore_index=True)
df_co= df_combined

df_combined



In [ ]:
column_names = df_combined.columns.tolist()
new_column_names = {
    column_names[0]: 'NewFeature1',
    column_names[1]: 'NewFeature2',
    column_names[2]: 'NewFeature3',
    column_names[3]: 'NewFeature4',
    column_names[4]: 'NewFeature5',
    column_names[5]: 'NewFeature6',
    column_names[6]: 'NewFeature7',
    column_names[7]: 'NewFeature8',
    column_names[9]: 'NewFeature9',

}

df_combined = df_combined.rename(columns=new_column_names)
df_combined.dtypes

In [ ]:
df_combined

In [ ]:
df_combined = df_combined.drop('NewFeature9', axis= 1)
df_co= df_combined

In [ ]:
df_co= df_combined

In [ ]:
from sklearn.model_selection import train_test_split


In [ ]:
train, test = train_test_split(df_combined, test_size=0.2, random_state=0)

In [ ]:
test

In [ ]:
train

In [ ]:
# , and convert it into a TensorFlow dataset.
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train, label="class")
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test, label="class")

In [ ]:
train

Following are the first five examples of the training dataset. Notice that
different columns represent different quantities. For example, how would you compare
the distance between *relationship* and *age*?

In [ ]:
# Print the first 5 examples.
train.head()

A Random Forest is trained as follows:

In [ ]:
# Train a Random Forest
model = tfdf.keras.RandomForestModel(num_trees=1000)
model.fit(train_ds)

The performance of the Random Forest model is:

In [ ]:
model_inspector = model.make_inspector()
out_of_bag_accuracy = model_inspector.evaluation().accuracy
print(f"Out-of-bag accuracy: {out_of_bag_accuracy:.4f}")

This is an expected accuracy value for Random Forest models on this dataset. It indicates that the model is correctly trained.

We can also measure the accuracy of the model on the test datasets:

In [ ]:
# The test accuracy is measured on the test datasets.
model.compile(["accuracy"])
test_accuracy = model.evaluate(test_ds, return_dict=True, verbose=0)["accuracy"]
print(f"Test accuracy: {test_accuracy:.4f}")

## Proximities

First, we inspect the number of trees in the model and the number of examples in the test datasets.

In [ ]:
print("The model contains", model_inspector.num_trees(), "trees.")
print("The test dataset contains", test.shape[0], "examples.")

In [ ]:
test

The method [predict_get_leaves()](https://www.tensorflow.org/decision_forests/api_docs/python/tfdf/keras/RandomForestModel) returns the index of the active leaf for each example and each tree.

In [ ]:
leaves = model.predict_get_leaves(test_ds)
print("The leaf indices:\n", leaves)

In [ ]:
print("The predicted leaves have shape", leaves.shape,
      "(we expect [num_examples, num_trees]")

In [ ]:
def compute_proximity(leaves, step_size=100):
  """Computes the proximity between each pair of examples.



  Returns:
    The example pair-wise proximity matrix of shape [n,n] with "n" the number of
    examples.
  """

  example_idx = 0
  num_examples = leaves.shape[0]
  t_leaves = np.transpose(leaves)
  proximities = []

  # Instead of computing the proximity in between all the examples at the same
  # time, we compute the similarity in blocks of "step_size" examples. This
  # makes the code more efficient with the the numpy broadcast.
  while example_idx < num_examples:
    end_idx = min(example_idx + step_size, num_examples)
    proximities.append(
        np.mean(
            leaves[..., np.newaxis] == t_leaves[:,
                                                example_idx:end_idx][np.newaxis,
                                                                     ...],
            axis=1))
    example_idx = end_idx
  return np.concatenate(proximities, axis=1)


proximity = compute_proximity(leaves)
print("The shape of proximity is", proximity.shape)

Here, `proximity[i,j]` is the proximity in between the example `i` and `j`.

The proximity matrix:

In [ ]:
proximity

The proximity matrix has several interesting properties, notably, it is symmetrical, positive, and the diagonal elements are all 1.

## Projection

Our first use of the proximity is to project the examples on the two dimensional plane.

If $\mathrm{prox} \in [0,1]$ is a proximity, $1 - \mathrm{prox}$ is a distance
between examples. Breiman proposes to compute the inner products of those distances, and to plot
the [eigenvalues](https://en.wikipedia.org/wiki/Eigenvalues_and_eigenvectors). See details
[here](https://www.stat.berkeley.edu/~breiman/RandomForests/cc_home.htm#scaling).

Instead, we will use the
[t-SNE](https://en.wikipedia.org/wiki/T-distributed_stochastic_neighbor_embedding)
which is a more modern way to visualize high-dimensional data.

**Note:** We use the [t-SNE's Scikit-learn implementation](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html).

In [ ]:
distance = 1 - proximity

t_sne = TSNE(
    # Number of dimensions to display. 3d is also possible.
    n_components=2,
    # Control the shape of the projection. Higher values create more
    # distinct but also more collapsed clusters. Can be in 5-50.
    perplexity=20,
    metric="precomputed",
    init="random",
    verbose=1,
    learning_rate="auto").fit_transform(distance)

In [ ]:
distance

In [ ]:
from scipy.cluster.hierarchy import linkage,dendrogram,fcluster
dict_linkage = {}
list_method= ['centroid','ward','median']

for k in list_method:
    dict_linkage[k] = linkage(
      y = distance,
      method = k,
      optimal_ordering = True)

In [ ]:
dict_linkage['ward']

In [ ]:
dict_linkage['median']

In [ ]:
dict_linkage['centroid']

In [ ]:
from matplotlib import pyplot as plt
fig = plt.figure(figsize=(25, 15))
dendrogram(
  Z = dict_linkage['centroid'],
  truncate_mode='level', p=2,
  count_sort = True,
  distance_sort = True,
  orientation = 'top',
  leaf_font_size = 15)
plt.show()

In [ ]:
cl = dict_linkage['centroid']
cl1 = dict_linkage['ward']
cl2 = dict_linkage['median']

In [ ]:
numclust =4

In [ ]:
fl_centroid = fcluster(cl,numclust,criterion='maxclust')
fl_centroid





In [ ]:
fl_median = fcluster(cl2,numclust,criterion='maxclust')
fl_median

In [ ]:
fl_ward = fcluster(cl1,numclust,criterion='maxclust')
fl_ward

In [ ]:
def mean(deneme, clusters_list):
    cluster_means = []
    for cluster_index in clusters_list:
        cluster_data = deneme.iloc[cluster_index]
        mean = np.mean(cluster_data, axis=0)
        cluster_means.append(mean)
    return cluster_means

In [ ]:
means_fl_centroid = mean(df_combined, fl_centroid)
means_fl_median = mean(df_combined, fl_median)
means_fl_ward = mean(df_combined, fl_ward)

In [ ]:
means_fl_ward

In [ ]:
means_fl_median


In [ ]:
means_fl_centroid

In [ ]:
def covariance(deneme, clusters):
    covariances = []
    for cluster_index in clusters:
        cluster_data = deneme.iloc[cluster_index]
        covariance = np.cov(cluster_data, rowvar=False)
        covariances.append(covariance)
    return covariances

In [ ]:
covariance_centroid = covariance(df_combined, fl_centroid)
covariance_centroid

In [ ]:
covariance_median = covariance(df_combined, fl_median)
covariance_median

In [ ]:
covariance_ward = covariance(df_combined, fl_ward)
covariance_ward

In [ ]:
from matplotlib import pyplot as plt
fig = plt.figure(figsize=(25, 15))
dendrogram(
  Z = dict_linkage['ward'],
  count_sort = True,
  truncate_mode='level', p=2,
  distance_sort = True,
  orientation = 'top',
  leaf_font_size = 15)
plt.show()

In [ ]:
from matplotlib import pyplot as plt
fig = plt.figure(figsize=(25, 15))
dendrogram(
  Z = dict_linkage['median'],
  truncate_mode='level', p=2,
  count_sort = True,
  distance_sort = True,
  orientation = 'top',
  leaf_font_size = 25)
plt.show()

In [ ]:
import pandas as pd
import numpy as np

# Number of data points
num_data_points = 2000

# Number of variables
num_variables = 8

# Probability of success for each variable (adjust as needed)
prob_success = 0.5

# Generate a DataFrame with random Bernoulli variables
data = np.random.choice([0, 1], size=(num_data_points, num_variables), p=[1 - prob_success, prob_success])
columns = [f'NewFeature{i+8}' for i in range(1, num_variables + 1)]
df_bernoulli = pd.DataFrame(data, columns=columns)

# Display the DataFrame
print(df_bernoulli.head())

In [ ]:
df_comb = pd.concat([df_ca, df_bernoulli], axis=1)
df_comb


In [ ]:
df_comb

The next plot shows a two-dimensional projection of the test example features. The color of the points
represent the label values. Note that the label values were not available to the model.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 8))
ax.grid(False)

# Color the points according to the label value.
colors = (test["class"] == "").map(lambda x: ["orange",'green'][x])
ax.scatter(
    t_sne[:, 0], t_sne[:, 1], c=colors, linewidths=0.5, marker="x", s=20)

With bernouli

In [ ]:
df_comb

In [ ]:
df_co

In [ ]:
from sklearn.cluster import KMeans, AgglomerativeClustering

kmeans = KMeans(n_clusters=4, random_state=0)
clusters_kmeans = kmeans.fit_predict(df_comb)
clusters_kmeans

In [ ]:
plt.subplot(132)
plt.scatter(df_comb['NewFeature4'], df_comb['NewFeature8'], c=clusters_kmeans, cmap='viridis')
plt.title("K-Means Clustering")

In [ ]:
def mean(deneme, clusters_list):
    cluster_means = []
    for cluster_index in clusters_list:
        cluster_data = deneme.iloc[cluster_index]
        mean = np.mean(cluster_data, axis=0)
        cluster_means.append(mean)
    return cluster_means

In [ ]:
kmeans_means = mean(df_comb, clusters_kmeans)

In [ ]:
kmeans_means

In [ ]:
kmeans_means[-1]

In [ ]:
kmeans_means[-501]

In [ ]:
kmeans_means[-1001]

In [ ]:
kmeans_means[-1501]

In [ ]:
kmeans_covariances

In [ ]:
import pandas as pd
import numpy as np

# Number of data points
num_data_points = 4000

# Number of variables
num_variables = 8

# Probability of success for each variable (adjust as needed)
prob_success = 0.5

# Generate a DataFrame with random Bernoulli variables
data = np.random.choice([0, 1], size=(num_data_points, num_variables), p=[1 - prob_success, prob_success])
columns = [f'NewFeature{i+8}' for i in range(1, num_variables + 1)]
df_bernoulli = pd.DataFrame(data, columns=columns)

# Display the DataFrame
print(df_bernoulli.head())

In [ ]:
df_co = pd.concat([df_co, df_bernoulli], axis=1)

In [ ]:
df_co

In [ ]:
df_combined = df_co

In [ ]:
from sklearn.model_selection import train_test_split


In [ ]:
train, test = train_test_split(df_combined, test_size=0.2, random_state=0)

In [ ]:
test

In [ ]:
train

In [ ]:
# , and convert it into a TensorFlow dataset.
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train, label="class")
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test, label="class")

In [ ]:
train

Following are the first five examples of the training dataset. Notice that
different columns represent different quantities. For example, how would you compare
the distance between *relationship* and *age*?

In [ ]:
# Print the first 5 examples.
train.head()

A Random Forest is trained as follows:

In [ ]:
# Train a Random Forest
model = tfdf.keras.RandomForestModel(num_trees=1000)
model.fit(train_ds)

The performance of the Random Forest model is:

In [ ]:
model_inspector = model.make_inspector()
out_of_bag_accuracy = model_inspector.evaluation().accuracy
print(f"Out-of-bag accuracy: {out_of_bag_accuracy:.4f}")

This is an expected accuracy value for Random Forest models on this dataset. It indicates that the model is correctly trained.

We can also measure the accuracy of the model on the test datasets:

In [ ]:
# The test accuracy is measured on the test datasets.
model.compile(["accuracy"])
test_accuracy = model.evaluate(test_ds, return_dict=True, verbose=0)["accuracy"]
print(f"Test accuracy: {test_accuracy:.4f}")

## Proximities

First, we inspect the number of trees in the model and the number of examples in the test datasets.

In [ ]:
print("The model contains", model_inspector.num_trees(), "trees.")
print("The test dataset contains", test.shape[0], "examples.")

In [ ]:
test

The method [predict_get_leaves()](https://www.tensorflow.org/decision_forests/api_docs/python/tfdf/keras/RandomForestModel) returns the index of the active leaf for each example and each tree.

In [ ]:
leaves = model.predict_get_leaves(test_ds)
print("The leaf indices:\n", leaves)

In [ ]:
print("The predicted leaves have shape", leaves.shape,
      "(we expect [num_examples, num_trees]")

In [ ]:
def compute_proximity(leaves, step_size=100):
  """Computes the proximity between each pair of examples.



  Returns:
    The example pair-wise proximity matrix of shape [n,n] with "n" the number of
    examples.
  """

  example_idx = 0
  num_examples = leaves.shape[0]
  t_leaves = np.transpose(leaves)
  proximities = []

  # Instead of computing the proximity in between all the examples at the same
  # time, we compute the similarity in blocks of "step_size" examples. This
  # makes the code more efficient with the the numpy broadcast.
  while example_idx < num_examples:
    end_idx = min(example_idx + step_size, num_examples)
    proximities.append(
        np.mean(
            leaves[..., np.newaxis] == t_leaves[:,
                                                example_idx:end_idx][np.newaxis,
                                                                     ...],
            axis=1))
    example_idx = end_idx
  return np.concatenate(proximities, axis=1)


proximity = compute_proximity(leaves)
print("The shape of proximity is", proximity.shape)

In [ ]:
proximity

The proximity matrix has several interesting properties, notably, it is symmetrical, positive, and the diagonal elements are all 1.

In [ ]:
distance = 1 - proximity

t_sne = TSNE(
    # Number of dimensions to display. 3d is also possible.
    n_components=2,
    # Control the shape of the projection. Higher values create more
    # distinct but also more collapsed clusters. Can be in 5-50.
    perplexity=20,
    metric="precomputed",
    init="random",
    verbose=1,
    learning_rate="auto").fit_transform(distance)

In [ ]:
distance

In [ ]:
from scipy.cluster.hierarchy import linkage,dendrogram,fcluster
dict_linkage = {}
list_method= ['centroid','ward','median']

for k in list_method:
    dict_linkage[k] = linkage(
      y = distance,
      method = k,
      optimal_ordering = True)

In [ ]:
dict_linkage['ward']

In [ ]:
dict_linkage['median']

In [ ]:
dict_linkage['centroid']

In [ ]:
from matplotlib import pyplot as plt
fig = plt.figure(figsize=(25, 15))
dendrogram(
  Z = dict_linkage['centroid'],
  truncate_mode='level', p=2,
  count_sort = True,
  distance_sort = True,
  orientation = 'top',
  leaf_font_size = 15)
plt.show()

In [ ]:
cl = dict_linkage['centroid']
cl1 = dict_linkage['ward']
cl2 = dict_linkage['median']

In [ ]:
numclust =4

In [ ]:
fl_centroid = fcluster(cl,numclust,criterion='maxclust')
fl_centroid





In [ ]:
fl_median = fcluster(cl2,numclust,criterion='maxclust')
fl_median

In [ ]:
fl_ward = fcluster(cl1,numclust,criterion='maxclust')
fl_ward

In [ ]:
def mean(deneme, clusters_list):
    cluster_means = []
    for cluster_index in clusters_list:
        cluster_data = deneme.iloc[cluster_index]
        mean = np.mean(cluster_data, axis=0)
        cluster_means.append(mean)
    return cluster_means

In [ ]:
means_fl_centroid = mean(df_combined, fl_centroid)
means_fl_median = mean(df_combined, fl_median)
means_fl_ward = mean(df_combined, fl_ward)

In [ ]:
means_fl_ward

In [ ]:
means_fl_median


In [ ]:
means_fl_centroid

In [ ]:
def covariance(deneme, clusters):
    covariances = []
    for cluster_index in clusters:
        cluster_data = deneme.iloc[cluster_index]
        covariance = np.cov(cluster_data, rowvar=False)
        covariances.append(covariance)
    return covariances

In [ ]:
covariance_centroid = covariance(df_combined, fl_centroid)
covariance_centroid

In [ ]:
covariance_median = covariance(df_combined, fl_median)
covariance_median

In [ ]:
covariance_ward = covariance(df_combined, fl_ward)
covariance_ward

In [ ]:
from matplotlib import pyplot as plt
fig = plt.figure(figsize=(25, 15))
dendrogram(
  Z = dict_linkage['ward'],
  count_sort = True,
  truncate_mode='level', p=2,
  distance_sort = True,
  orientation = 'top',
  leaf_font_size = 15)
plt.show()

In [ ]:
from matplotlib import pyplot as plt
fig = plt.figure(figsize=(25, 15))
dendrogram(
  Z = dict_linkage['median'],
  truncate_mode='level', p=2,
  count_sort = True,
  distance_sort = True,
  orientation = 'top',
  leaf_font_size = 25)
plt.show()